# Building a Simple PC with RPCircuits

In [1]:
using RPCircuits, Random

Random.seed!(42)

TaskLocalRNG()

The easyest way to build a PCs with RPCircuits uses a bottom-up strategy, definin first the `leaf` nodes, then `product` nodes, and lastly `sum` nodes. 

As an example, we start building a simple Circuit with 8 indicator (leafs), 4 products and 1 sum node.

![basic](spn_basic.png)

First we have the foundation of the PC, the **leaf** nodes. In this case the variables **a**, **b** and their respective negations **na** and **nb**. To declare an indicator leaf node using RPCircuits, we call the function `Indicator()`, that needs **two** arguments. The first argument simply refers to the variable's `scope` (the indices of the variables associated with the Indicator node). The second argument, corresponds to the `value` such that the Indicator node outputs `true` (if `value = 1.0`, the Indicator node outputs `true` only when its input is `1.0`). Using the packag we have

In [3]:
a, na, b, nb = Indicator(1, 1.0), Indicator(1, 0.), Indicator(2, 1.), Indicator(2, 0.)

(indicator 1 1.0, indicator 1 0.0, indicator 2 1.0, indicator 2 0.0)

Next, we have the second layer with 4 **product** nodes. To define a product `P`, we only need to call the function `Product(v)`, where `v`  is the vector containing all children of `P`. Hence, we can build the four products of our PC by

In [4]:
P1, P2, P3, P4 = Product([a,b]), Product([a,nb]), Product([na,b]), Product([na,nb])

(* 1 2, * 1 2, * 1 2, * 1 2)

At last, we have the **sum** node. To define a sum  node `S`, we have to call the function `Sum(v, w)`, where `v` is the vector of children of `S`; and `w` is the vector of corresponding weights. This can easily be done by

In [5]:
S = Sum([P1, P2, P3, P4], [0.4, 0.3, 0.2, 0.1])

Circuit with 9 nodes (1 sum, 4 products, 4 leaves) and 2 variables:
  1 : + 1 0.4 2 0.3 3 0.2 4 0.1
  2 : * 1 2
  3 : * 1 2
  4 : indicator 1 0.0
  5 : * 1 2
  6 : indicator 2 0.0
  7 : * 1 2
  8 : indicator 2 1.0
  9 : indicator 1 1.0

Hence, we have the circuit
![basic](spn_example.png)

To see the `scope` of a circuit rooted at a node `C`, we can type `scope(C)`. Therefore, the scope of our circuit is

In [7]:
scope(S)

BitSet with 2 elements:
  1
  2

Using RPCircuits, it is possible to randomly sample complete configurations of the variables associated with a circuit `C`. We can do this using the function `rand(C)`, that creates a sample according to the probability defined by the PC.

In [10]:
sample = rand(S)

2-element Vector{Float64}:
 0.0
 1.0

Passing a positive integer `N` to `rand(C, N)`, creates `N` random samples.

In [8]:
N = 1_000
D = rand(S, N)

1000×2 Matrix{Float64}:
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 ⋮    
 1.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  0.0
 1.0  1.0
 0.0  0.0
 1.0  1.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0

With the function `NLL(S, D)`, we have the `Negative Log-Likelihood` of the PC `S` w.r.t the dataset `D`.

In [11]:
println("Original model NLL = ", NLL(S, D))

Original model NLL = 1.2905776805822866


Suppose that we have an initial circuit `S_em = Sum([P1, P2, P3, P4], [0.25, 0.25, 0.25, 0.25])` and we want to learn the function `S` (such that configurations `(a,b)`, `(a,nb)`, `(na,b)` and `(na, nb)` have respective probabilities `0.4`, `0.3`, `0.2` and `0.1`). Firstly, we can check the initial `NLL` of our model `S_em` in relation to the dataset `D`.

In [14]:
S_em = Sum([P1, P2, P3, P4], [0.25, 0.25, 0.25, 0.25])

println("Initial NLL = ", NLL(S_em, D))

Initial NLL = 1.3862943611198644


Now, we can pass both our circuit `S_em` and the dataset `D` as an input to the `EM` algorithm (Expectation-Maximization algorithm, more to know about it [here][murphy]). To do this, we first define the learner `L = SEM(S)`. Then, we have `m` calls of the `update` function, for `m` iterations of the `EM` algorithm.

[murphy]: https://probml.github.io/pml-book/book1.html

In [17]:
L_em = SEM(S_em)

for i = 1:50
    update(L_em, D)
end

At last, we can apply the `NLL` function another time, to see the improvement obtained by the learning process.

In [18]:
println("Final NLL = ", NLL(S_em, D))

Final NLL = 1.2891629841331291


Similarly, we can use the Gradient Descent algorithm (more to know about it [here][murphy]) to learn a circuit `S_grad` w.r.t the dataset `D`. In this process, we have a sligthly different approach, because we initalize the sum-weights near zero (more to know about it [here][trapp]).

[murphy]: https://probml.github.io/pml-book/book1.html
[trapp]: https://arxiv.org/abs/1905.08196

In [30]:
using Distributions

# Incialize weigths close to zero
init_weights = 0.1
w = Distributions.rand(Distributions.truncated(Distributions.Normal(0, init_weights), 0, Inf), 4)

S_grad = Sum([P1, P2, P3, P4], w)

L_grad = GRAD(S_grad);

Since the circuit `S_grad` is not normalized, we need to compute its **normalizing constant**. We do this by using the function `log_norm_const!` that outputs the `log` of the normalizing constant (`norm_V` is an auxiliary vector in the process of computing `norm_const`).

In [43]:
# Auxiliary vector for the computation of the normalizing constant of the circuit
norm_V = Vector{Float64}(undef, length(S))

# Normalizing Constant of the circuit
norm_const = RPCircuits.log_norm_const!(norm_V,L_grad.circ.C)

-1.6823024104143856

Now, it is possible to obtain the real `NLL` of `S_grad` w.r.t `D` by adding `nomr_const` to `NLL(S_grad, D)`

In [48]:
println("Initial NLL = ", NLL(S_grad, D) + norm_const)

Initial NLL = 1.3237494967673191


Finally, we can apply the Gradient Descent algorithm to `S_grad` w.r.t `D` and then see the improvement obtained by the learning process.

In [50]:
for i = 1:1_000
    update(L_grad, D; learningrate=0.01)
end

# New normalizing constant of the circuit
norm_const = RPCircuits.log_norm_const!(norm_V,L_grad.circ.C) 

println("Final NLL = ", NLL(S_grad, D) + norm_const)

Final NLL = 1.3118771877259712
